In [ ]:
%%HTML
<!-- Make fonts readable at 1024x768 -->
<style>
.rendered_html { font-size:1.em; }
</style>

In [ ]:
# Imports and matplotlib configuration
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np
import scipy
import pandas as pd
import pymc3 as pm
print('Running PyMC3 v{}'.format(pm.__version__))
import theano.tensor as T
from matplotlib import animation
from ipywidgets import interact
from IPython.display import display
from style import *

## Universidad Austral de Chile

# INFO337 - Herramientas estadísticas para la investigación

# Markov Chain Monte Carlo

### A course of the masters in informatics program

### https://github.com/magister-informatica-uach/INFO337


***
<a id="index"></a>

# Contents
***

1. [The Bayesian setting](#section1)
1. [Markov Chain Monte Carlo](#section2)
1. [Tutorial: Linear regression](#section3)
1. [Tutorial: GMM](#section4)
1. [Tutorial: Multilabel logistic regression](#section5)


### References

1. Davidson-Pilon, "[Bayesian methods for hackers](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers)", *Addison Wesley*, 2016, **Chapter 2 and 3**



***
<a id="section1"></a>

# The Bayesian setting
***

Reminder from previous classes:
- **Probability:** Represents how believable an event is
     - How confident we are that the event occurs
- Our belief of a certain event $A$ is the **prior probability** $p(A)$
- We collect evidence $X$ to **update** our belief on $A$ forming a **posterior** $p(A|X)$
- How? Bayes Theorem
$$
p(A|X) = \frac{p(X|A)p(A)}{p(X)} \propto p(X|A)p(A)
$$
- In general the larger the amount of evidence $N$ the less influence the prior has

***

Model/parameter estimation:
- Maximum likelihood (MLE): point estimate
- Maximum a posterior (MAP): point estimate plus prior
- Bayes: full posterior distribution

One more time: Bayes Theorem + law of total probability:
$$
p(A|X) = \frac{p(X|A)p(A)}{p(X)} = \frac{p(X|A)p(A)}{\int p(X, A) dA} = \frac{p(X|A)p(A)}{\int p(X|A)p(A) dA}
$$

- We propose the **prior** and a **likelihood**: our assumptions on the data and parameter distributions
- The **evidence** is ... usually intractable 
    - We are integrating on all the possible values of the parameters, remember GMM?
- Are we done? Luckily no
    - Use priors/likelihoods so that posterior is analytical (conjugates)
    - Approximate inference (Variational Bayes)
    - **Today:** Monte-Carlo Markov Chain

### [The Bayesian landscape](http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter3_MCMC/Ch3_IntroMCMC_PyMC3.ipynb#The-Bayesian-landscape)

- In the following example we draw two dimensional Poisson distributed data
- We consider exponential priors for the $\lambda$
- This is essentially a two dimensional surface in parameter space
- The prior sets the initial shape 
- The observations warp the surface
- To find the best parameters we need to explore this possibly high-dim space

In [ ]:
np.random.seed(0)
fig, ax = plt.subplots(1, 2, figsize=(7, 4))
def update(rseed, N):
    np.random.seed(rseed); ax[0].cla(); ax[1].cla();
    lambda_1_true = 1; lambda_2_true = 3
    data = np.concatenate([scipy.stats.poisson.rvs(lambda_1_true, size=(N, 1)),
                           scipy.stats.poisson.rvs(lambda_2_true, size=(N, 1))
                          ], axis=1)

    x = y = np.linspace(.01, 5, 100)
    likelihood_x = np.array([scipy.stats.poisson.pmf(data[:, 0], _x)
                            for _x in x]).prod(axis=1)
    likelihood_y = np.array([scipy.stats.poisson.pmf(data[:, 1], _y)
                            for _y in y]).prod(axis=1)
    L = np.dot(likelihood_x[:, None], likelihood_y[None, :])


    exp_x = scipy.stats.expon.pdf(x, loc=0, scale=3)
    exp_y = scipy.stats.expon.pdf(x, loc=0, scale=10)
    M = np.dot(exp_x[:, None], exp_y[None, :])

    im = ax[0].imshow(M, interpolation='none', origin='lower',
                    cmap=plt.cm.jet, extent=(0, 5, 0, 5))
    ax[0].scatter(lambda_2_true, lambda_1_true, c="k", s=50, edgecolor="none")
    ax[0].set_xlim(0, 5); ax[0].set_ylim(0, 5)
    ax[0].set_title("Prior Landscape")

    im = ax[1].imshow(M * L, interpolation='none', origin='lower',
                    cmap=plt.cm.jet, extent=(0, 5, 0, 5))

    ax[1].scatter(lambda_2_true, lambda_1_true, c="k", s=50, edgecolor="none")
    ax[1].set_title("Landscape warped \nby %d data observation." % N)
    ax[1].set_xlim(0, 5); ax[1].set_ylim(0, 5);
interact(update, N=SelectionSlider_nice(options=[1, 10, 100]),
         rseed=IntSlider_nice(min=0, max=100));

[&larr; Go back to the index](#index)

***

<a id="section2"></a>

# Markov Chain Monte Carlo (MCMC)
***

- Monte carlo methods obtain numerical results via repeated random sampling
- Example: Monte-carlo integration
    - Non-deterministic approach to compute definite integral
    - Draw uniform random samples (N-d square)
    - Check which of those are inside your function and which are not
    - The search is naive

In [ ]:
def func(x, y):
    return (x-0)**2 + (y-0)**2 -1. <= 0.
x = np.linspace(0, 1, num=1000); X, Y = np.meshgrid(x, x)
fig, ax = plt.subplots(figsize=(6, 5))

def update(N, rseed):
    np.random.seed(rseed);
    ax.cla(); ax.contourf(X, Y, func(X, Y), cmap=plt.cm.Reds); ax.set_aspect('equal')
    xr = np.random.rand(N, 2)
    if N < 1000000:
        ax.scatter(xr[:, 0], xr[:, 1], s=1, alpha=0.5)
    N_inside = len(np.where(func(xr[:, 0], xr[:, 1]))[0])
    print("%0.4f" %(4.*N_inside/N))

interact(update, N=SelectionSlider_nice(options=[1, 10, 100, 1000, 10000, 100000, 1000000]), 
         rseed=IntSlider_nice(min=0, max=100));

- Now consider a system, i.e. a set of states that evolve in time (steps)
- A system can be modeled with a **Markov Chain**: stochastic process for sequences
- Given that it fits the **Markov property**
    - The conditional probability of future states depends only on the present state
    - $p(X_n|X_{n-1},\ldots, X_0) = p(X_n|X_{n-1})$
    - We only need knowledge from one state to move to another

## MCMC 

- MCMC is a family of algorithms to generate random samples from a probability distribution
- We sample (Monte-carlo) but every sample depends on the previous one (Markov chain)
    - This way we search the space in a less naive way
    - Random walk with a generated **step**
    - We will review some algorithms that select the step
- A collection of samples is called a **trace**
    - From the trace  we can approximate the posterior we are looking for
    - ... if the chain **converged**

## Metropolis Hastings
- Random walker that moves on all dimensions simultaneously
- A candidate step is drawn from an arbitrary but symmetric distribution $x^{new} \sim g(x^{new}|x_t)$
- We accept the step if $f(x^{new})/f(x^t)$ is equal or larger than a certain threshold
- $f(\cdot)$ needs only to be proportional to the target distribution (evidence is canceled in the ratio)
- Repeat many times until convergence


## Hamiltonian Monte-Carlo
- Family of step proposing methods that use momentum (derivatives)
- Only for continuous variables
- Cost more than MH (single iteration) but they require less iterations
- http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html

***
# Probabilistic programming
***

- PP: Doing statistics (Bayesian inference) using the tools of computer science 
- PP languages: unify general purpose programming with probabilistic modeling
- Python friendly PP frameworks/libraries:
    - [PyMC3](https://docs.pymc.io/notebooks/getting_started.html): Black-box VI, MH, Gibbs, NUTS sampler. Uses theano
    - [PyStan](https://pystan.readthedocs.io/en/latest/): Python interface for [Stan platform](http://mc-stan.org/)
    - [Edward](http://edwardlib.org/): Black-box VI, neural networks. Uses tensorflow
    - [emcee](http://dfm.io/emcee/current/): Pure python implementation of the Affine invariant MCMC ensemble sampler
    - http://mattpitkin.github.io/samplers-demo/pages/samplers-samplers-everywhere/

<a href="https://arxiv.org/abs/1809.10756"><img src="img/PP.png"></a>

PP run in two directions!

[&larr; Go back to the index](#index)

<a id="section3"></a>
***
# A PyMC3 tutorial
***

In this tutorial we will review how to do
1. Model definition
1. Fitting
1. Convergence checks
1. Posterior analysis
with PyMC3

We start with an example of Bayesian linear regression (class 2)
- Gaussian noise with variance $\sigma^2$
- One independent variables: $X$
- Three parameters $\beta$: intercept plus one coefficient per covariate

$$
Y = \beta X + \epsilon 
$$

$$
\epsilon \sim \mathcal{N}(0, \sigma^2)
$$

$$
Y \sim \mathcal{N}(\mu, \sigma^2)
$$

$$
\mu = \beta_0 + \beta_1 X
$$

Credit: https://docs.pymc.io/notebooks/getting_started.html

In [ ]:
np.random.seed(0)
beta_true, sigma_true, N = [1, 2.5], 1., 30
X = np.random.randn(N)
Y = beta_true[0] + beta_true[1]*X +  np.random.randn(N)*sigma_true

fig, ax = plt.subplots(figsize=(7, 4))
ax.scatter(X, Y)
ax.set_xlabel('X'); ax.set_ylabel('Y'); 

### Model specification
- First we instantiate a model from `pm.Model()`
- This creates a context for our model
- Within this context we will set priors, likelihood, etc


In [ ]:
with pm.Model() as my_model:
    # Priors
    beta = pm.Normal(name='beta', mu=0, sd=10, shape=2)
    sigma = pm.HalfNormal('sigma', sd=1, testval=np.std(data))

- We have set two stochastic variables
- These are the priors for $\beta$ and $\sigma$
- Check available distributions [here](https://docs.pymc.io/api/distributions.html)
- For $\beta$ we used a normal prior. The constructor for normal is

`pm.Normal(name='beta', mu=0, sd=10, shape=3)`
where
1. name (string): Unique identifier, in this case 'beta'
1. mu and sd (floats): mean and standard deviation of the normal distribution in this case 0 and 10, respectively
1. shape: specifies the dimensionality, in this case we create 3 univariate normals


- For $\sigma$ we use a half-normal prior
    - $\sigma$ is non-negative so we have to use a non-negative prior, e.g. gamma
    - Other option is to use a bounded distribution
    - One can create arbitrary bounded distributions with
    
    `x = pm.Bound(pm.Normal, lower=0.0)('x', mu=1.0, sd=3.0)`



- We can give initial values for the variables using `testval`


To continue working in this context we use 

`with my_model:` 

In [ ]:
with my_model:
    # Likelihood
    Y_obs = pm.Normal('Y_obs', mu=beta[0] + beta[1]*X, sd=sigma, observed=Y)

- To specify the likelihood we select a distribution and give the "special" argument `observed`
- This corresponds to the data
- It can be a numpy ndarray or pandas data frame
- By giving `beta` and `sigma` as parameter for `Y_obs` we automatically create a parent-child relation



In [ ]:
print(my_model.free_RVs)
print(my_model.deterministics)
print(my_model.observed_RVs)
my_model

### Model fitting

In PyMC3 we can do MAP to get point estimates, VI to do approximate inference and MCMC for posterior sampling

- MAP and VI follow an optimization approach. 
- They are generally faster than MCMC but return less information 
    - point estimate with no uncertainty
    - approximate factorized distribution (only continuous variables)
- MAP and VI can be used to find reasonable initial states for MCMC
- For very complex model and large number of observations we may not be able to do MCMC at all 

**MAP estimate**

In [ ]:
with my_model:
    map_estimate = pm.find_MAP(progressbar=True)
map_estimate

### MCMC sampling

- To do MCMC first you have to specify a step method (MH, Gibbs, NUTS, etc)
- PyMC3 have very good default options
    - No-U-Turn sampler is the default option for continuous parameters
    - MH is the default for discrete parameters
- Use diagnostics to check the convergence of the chains
- Check posteriors with traceplots

`sample(draws=500, step=None, init='auto', n_init=200000, start=None, trace=None, chain_idx=0, chains=None, cores=None, tune=500, nuts_kwargs=None, step_kwargs=None, progressbar=True, model=None, random_seed=None, live_plot=False, discard_tuned_samples=True, live_plot_kwargs=None, compute_convergence_checks=True, use_mmap=False, **kwargs)`

In [ ]:
with my_model:
    # draw 500 posterior samples
    trace = pm.sample(draws=2000, start=map_estimate)

In [ ]:
pm.traceplot(trace, figsize=(9, 6), combined=True);

In [ ]:
pm.plot_posterior(trace, figsize=(9, 6), kde_plot=True);

In [ ]:
df_trace = pm.trace_to_dataframe(trace)
pd.plotting.scatter_matrix(df_trace, diagonal='kde', figsize=(8, 6))
plt.tight_layout()

In [ ]:
pm.summary(trace).round(2)

In [ ]:
X_plot = np.ones(shape=(1000, 2))
X_plot[:, 1] = np.linspace(np.amin(X), np.amax(X), num=1000)
fig, ax = plt.subplots(figsize=(7, 4))
ax.scatter(X, Y, c='k', label='data');
ax.set_xlabel('X'); ax.set_ylabel('Y'); 

beta_mean, beta_std = np.mean(trace['beta'], axis=0), np.std(trace['beta'], axis=0)
ax.plot(X_plot[:, 1], np.dot(X_plot, beta_mean), lw=3, label='Inferred model') 
ax.fill_between(X_plot[:, 1], np.dot(X_plot, beta_mean-2*beta_std), 
                np.dot(X_plot, beta_mean+2*beta_std), alpha=0.5)
ax.plot(X_plot[:, 1], np.dot(X_plot, beta_true), lw=3, label='True model')
plt.legend();

The Gelman-Rubin diagnostic tests for lack of convergence 
- Compares the variance between multiple chains to the variance within each chain.
- Values greater than one indicate that one or more chains have not yet converged.

In [ ]:
pm.gelman_rubin(trace)

[&larr; Go back to the index](#index)

<a id="section4"></a>
***
# Second example: Mixture of Gaussians
***
- In this example we will try to infer the parameters of a mixture of two 1d Gaussians
- Let's cerate some data

Ref: http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter3_MCMC/Ch3_IntroMCMC_PyMC3.ipynb

In [ ]:
mu_true = [-1, 2]
std_true = [2, 0.75]
p_true, N = 0.4, 200
p = np.array([p_true, 1-p_true])
np.random.seed(0)
data = np.concatenate((scipy.stats.norm(loc=mu_true[0], scale=std_true[0]).rvs(size=int(p[0]*N)),
                       scipy.stats.norm(loc=mu_true[1], scale=std_true[1]).rvs(size=int(p[1]*N))))

fig, ax = plt.subplots(figsize=(9, 3))
ax.hist(data, bins=20, alpha=0.8, density=True);
x_plot = np.linspace(np.amin(data), np.amax(data), num=1000)
for k in range(2):
    ax.plot(x_plot, p[k]*np.exp(-0.5*(x_plot - mu_true[k])**2/std_true[k]**2)/(np.sqrt(2.*np.pi)*std_true[k]), 'k--')

**Model specification**

We create priors for the center, dispersion and weights of the Gaussians

Ref: https://docs.pymc.io/notebooks/gaussian_mixture_model.html

In [ ]:
with pm.Model() as model:
    # Prior on concentration
    p = pm.Dirichlet('p', a=np.array([0.1, 0.1]), shape=2)
    p_min_potential = pm.Potential('p_min_potential', T.switch(T.min(p) < .1, -np.inf, 0))
    z = pm.Categorical("z", p, shape=data.shape[0])
    # Prior on standard deviations
    sds = pm.Uniform("sds", lower=0, upper=100, shape=2)
    # Prior on means
    centers = pm.Normal("centers", 
                        mu=np.array([-1, 1]), 
                        sd=np.array([10, 10]), 
                        shape=2)
    order_means_potential = pm.Potential('order_means_potential',
                                         T.switch(centers[1]-centers[0] < 0, -np.inf, 0))
    
    center_i = pm.Deterministic('center_i', centers[z])
    sd_i = pm.Deterministic('sd_i', sds[z])
    
    # Likelihood
    observations = pm.Normal("obs", mu=center_i, sd=sd_i, observed=data)

Deterministic variables allow us to track custom variables in the traces

`pm.Deterministic('name', var)`

Potentials allow us to add an arbitrary factor the the likelihood

`pm.Potential('name', var)`

In this case we add potentials to
- Penalize solutions with  empty clusters
- Forcing that the "first" gaussian is always to the left (remember z flipping)

**Model fitting**

Let's try MAP and then MCMC as before

In [ ]:
with model:
    start = pm.find_MAP()
print(start['centers'])
print(start['sds'])
print(start['p'])

In [ ]:
with model:
    step1 = pm.Metropolis(vars=[p, sds, centers])
    # step1 = pm.NUTS(vars=[p, sds, centers])
    step2 = pm.ElemwiseCategorical(vars=[z], values=[0, 1])
    # step2 = pm.CategoricalGibbsMetropolis(vars=[z])
    trace = pm.sample(draws=100, step=[step1, step2], tune=0, chains=2, cores=2, start=None)

 - `step`: In this case we have specified the step functions for each variable
 - Burn-in period `tune`: N first steps of the chain are discarded from the trace to build posteriors from the converged zone
 - `chains`: We can also specify the amount of chains and cores 

In [ ]:
pm.traceplot(trace, figsize=(9, 6), combined=True, varnames=['p', 'centers', 'sds']);

In [ ]:
print(pm.gelman_rubin(trace)['centers'])
print(pm.gelman_rubin(trace)['sds'])
print(pm.gelman_rubin(trace)['p'])

In [ ]:
pm.plots.autocorrplot(trace=trace, figsize=(9, 8), varnames=['centers', 'sds', 'p']);

- A chain that is exploring the space well will exhibit very high autocorrelation. 
- Low autocorrelation is a sufficient condition for converged MCMC

In [ ]:
pm.summary(trace, varnames=['centers', 'sds', 'p']).round(2)

In [ ]:
pm.plot_posterior(trace, figsize=(9, 7), kde_plot=True, varnames=['centers', 'sds', 'p']);

### Posterior predictive checks (PPC)

Another way to validate a model is to generate data from the posterior draws and compare with the original distribution

This is done with `pm.sample_posterior_predictive` or `pm.sample_ppc` depending on your PyMC version

In [ ]:
ppc = pm.sample_ppc(samples=1000, trace=trace, model=model)
print(ppc.keys())
x_plot = np.linspace(np.amin(data), np.amax(data), num=1000)
from sklearn.neighbors.kde import KernelDensity

fig, ax = plt.subplots(figsize=(9, 3))
ax.hist(data, bins=20, alpha=0.8, density=True);
mean_score = 0.0
for i in range(100):
    kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(ppc['obs'][i, :].reshape(-1, 1))
    score = np.exp(kde.score_samples(x_plot.reshape(-1, 1)))
    mean_score += score
    ax.plot(x_plot, score, 'k', alpha=0.05)
ax.plot(x_plot, mean_score/100, 'k', lw=4)    

### [Useful tips for MCMC](http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter3_MCMC/Ch3_IntroMCMC_PyMC3.ipynb#Useful-tips-for-MCMC)

- Intelligent starting values
- Choose your priors well!

[&larr; Go back to the index](#index)

<a id="section5"></a>
***
# Third example: Multilabel logistic (softmax) regression
***

In [ ]:
!wget -nc http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -P data

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

df = pd.read_table("data/iris.data", delimiter=",",  na_values="?",
                   names= ["sepal length", "sepal width", "petal length", "petal width", "class"])
# numeric labels
labels = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
def compare(x):
    for k in range(3):
        if x == labels[k]:
            return k
df["class"] = df["class"].apply(compare)
# To numpy
label = df.iloc[:, -1].values
data =df.iloc[:, 0:4].values
# Create train and test index
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3)
train_idx, test_idx = next(sss.split(data, label))

In [ ]:
from theano import shared
X = shared(data[train_idx, :])
Y = shared(label[train_idx])

with pm.Model() as my_model:
    # Priors
    alpha = pm.Normal(name='alpha', mu=0, sd=10, shape=(3, ))
    beta = pm.Normal(name='beta', mu=0, sd=10, shape=(4, 3))
    #sigma = pm.HalfNormal('sigma', sd=1)
    p = T.nnet.softmax(X.dot(beta) + alpha)
    label_obs = pm.Categorical('labels_obs', p=p, observed=Y)

In this case we will try ADVI to get a starting point

**Automatic differentiation Variational inference (ADVI)**

[PyMC Variational API](https://docs.pymc.io/notebooks/variational_api_quickstart.html)

1. Select 'advi' or 'fullrank_advi' as method to fit the model
1. Choose number of iterations
1. Get a trace from the fitted model
1. Study the convergence of the objective function and traced parameters
1. Inspect traces and posteriors with diagnostic plot

In [ ]:
with my_model:    
    # inference = pm.ADVI()
    inference = pm.FullRankADVI()
    # inference = pm.SVGD(n_particles=500, jitter=1)
    approx = pm.fit(method=inference, n=30000, 
                    obj_optimizer=pm.adam(learning_rate=0.001))
    train_probs = approx.sample_node(p)
    test_probs = approx.sample_node(p, more_replacements={X: data[test_idx, :]})
    advi_trace = approx.sample(draws=5000)     

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(approx.hist, alpha=.3); ax.set_yscale('log')

In [ ]:
test_acc, train_acc = [], []
for i in range(10):
    test_acc.append(np.mean(test_probs.argmax(-1).eval() ==  label[test_idx]))
    train_acc.append(np.mean(train_probs.argmax(-1).eval() ==  label[train_idx]))
train_acc = np.array(train_acc)
test_acc = np.array(test_acc)
print("Train: %f %f" % (np.mean(train_acc), np.std(train_acc)))
print("Test: %f %f" % (np.mean(test_acc), np.std(test_acc)))


In [ ]:
df_trace = pm.trace_to_dataframe(advi_trace)
pd.plotting.scatter_matrix(df_trace, diagonal='kde', figsize=(9, 8));